In [15]:
import pandas as pd
import os
print(os.getcwd())

c:\Users\mr074\OneDrive\Documents\AllProgram\bangla-emotion-detection-ann


In [16]:
df = pd.read_csv(r'c:\Users\mr074\OneDrive\Documents\AllProgram\bangla-emotion-detection-ann\bangla-emotions-datasets\Train.csv')
df.head()


,ID,Data,Love,Joy,Surprise,Anger,Sadness,Fear,Topic,Domain,is_admin
0,5454,লকাল বাস ভালো এটা থেকে,0,0,0,0,1,0,Travel,Youtube,False
1,22549,কত অভিজানই তো চলে কিন্তু ওয়াসার পানির অভিজান ক...,0,0,0,0,1,0,Politics,Youtube,False
2,7033,বিয়ের মহল ছেড়ে তিনি বিস্রাম নিতে চলে যান (৬ ...,0,0,0,1,0,0,Personal,Facebook,False
3,21114,চাচাজি তো কেবল মাকে ধর্ষণ করেছেন,0,0,0,0,1,0,Education,Facebook,False
4,23683,সত্যিকার মানুষ তারাই ভাই,0,1,0,0,0,0,Personal,Youtube,False


In [17]:
df.drop(columns=['ID','Topic','Domain','is_admin'], inplace=True)

In [18]:
df.head()

,Data,Love,Joy,Surprise,Anger,Sadness,Fear
0,লকাল বাস ভালো এটা থেকে,0,0,0,0,1,0
1,কত অভিজানই তো চলে কিন্তু ওয়াসার পানির অভিজান ক...,0,0,0,0,1,0
2,বিয়ের মহল ছেড়ে তিনি বিস্রাম নিতে চলে যান (৬ ...,0,0,0,1,0,0
3,চাচাজি তো কেবল মাকে ধর্ষণ করেছেন,0,0,0,0,1,0
4,সত্যিকার মানুষ তারাই ভাই,0,1,0,0,0,0


In [19]:
import re

def clean_text(text):
    text = str(text)
    text = re.sub(r'[^\u0980-\u09FF\u09E6-\u09FF\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

In [20]:
df['messages'] = df.Data.apply(clean_text)

In [21]:
df.drop(columns=['Data'], inplace=True)

In [22]:
df

,Love,Joy,Surprise,Anger,Sadness,Fear,messages
0,0,0,0,0,1,0,লকাল বাস ভালো এটা থেকে
1,0,0,0,0,1,0,কত অভিজানই তো চলে কিন্তু ওয়াসার পানির অভিজান ক...
2,0,0,0,1,0,0,বিয়ের মহল ছেড়ে তিনি বিস্রাম নিতে চলে যান ৬ ন...
3,0,0,0,0,1,0,চাচাজি তো কেবল মাকে ধর্ষণ করেছেন
4,0,1,0,0,0,0,সত্যিকার মানুষ তারাই ভাই
...,...,...,...,...,...,...,...
18415,0,1,0,0,0,0,সমস্যা নাই ইনশাআল্লাহ আমরা একদিন ক্রিকেট কে শা...
18416,0,0,0,0,1,0,খানকিরপুলা তর মাকে গিয়ে বল দেহো দিলা মনটা দিলানা
18417,0,0,1,0,0,0,ভাই চেয়ার কই
18418,0,0,0,0,1,0,সবকয়টা তো একই গোয়ালের গাই


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    ngram_range=(1,2),
    max_features=3000
)

X = vectorizer.fit_transform(df.messages).toarray()
y = df.drop('messages',axis=1)

In [24]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(18420, 3000))

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)


In [26]:
from tensorflow.keras.utils import to_categorical
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
y_train_cat = y_train_cat.reshape(len(y_train_cat),6*2)
y_test_cat = y_test_cat.reshape(len(y_test_cat),6*2)

In [27]:
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(128, input_shape=(X_train.shape[1],), activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(y_train_cat.shape[1], activation='sigmoid')
])

model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

model.summary()

c:\Users\mr074\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 128)            │       384,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 12)             │           780 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 393,164 (1.50 MB)

 Trainable params: 393,164 (1.50 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
model.fit(X_train, y_train_cat, epochs=15, batch_size=32,validation_split=0.1)

Epoch 1/15
415/415 ━━━━━━━━━━━━━━━━━━━━ 13s 14ms/step - accuracy: 0.6148 - loss: 251.6519 - val_accuracy: 0.8005 - val_loss: 765.8848
Epoch 2/15
415/415 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.7932 - loss: 1712.3431 - val_accuracy: 0.8005 - val_loss: 2838.5022
Epoch 3/15
415/415 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.7932 - loss: 3937.7000 - val_accuracy: 0.8005 - val_loss: 5253.0884
Epoch 4/15
415/415 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.7932 - loss: 6682.1865 - val_accuracy: 0.8005 - val_loss: 8063.5459
Epoch 5/15
415/415 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.7932 - loss: 9075.1709 - val_accuracy: 0.8005 - val_loss: 9899.6074
Epoch 6/15
415/415 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.7932 - loss: 11091.3164 - val_accuracy: 0.8005 - val_loss: 12540.3809
Epoch 7/15
415/415 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - accuracy: 0.7932 - loss: 13978.4590 - val_accuracy: 0.8005 - val_loss: 15740.2822
Epoch 8/15
415/415 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/ste

In [35]:
model.predict([X_train[0]])

c:\Users\mr074\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\models\functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_4
Received: inputs=('Tensor(shape=(32,))',)
  warnings.warn(msg)


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(32,), dtype=float32) with name 'keras_tensor_4' and path ''. Expected shape (None, 3000), but input has incompatible shape (32,)[0m

Arguments received by Sequential.call():
  • inputs=('tf.Tensor(shape=(32,), dtype=float32)',)
  • training=False
  • mask=('None',)
  • kwargs=<class 'inspect._empty'>